In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision
# Örnek bir transform tanımla (istediğin şekilde düzenleyebilirsin)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Veri setini yükle
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Kidney-Cancer', transform=transform)
BATCH_SIZE = 16
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

# Eğitim verisini %10 doğrulama için ayır
val_size = int(0.1 * len(train_set))
train_size -= val_size  # Geriye kalan kısım eğitim verisi olacak

train_set, val_set = torch.utils.data.random_split(train_set, [train_size, val_size])

# DataLoader'ları oluştur (batch_size'i ihtiyacınıza göre ayarlayabilirsin)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model.classifier  = nn.Linear(1280,len(dataset.classes))

saved_weights = "./transfer-learning-3.pth"

model.load_state_dict(torch.load(saved_weights))

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 162MB/s]


<All keys matched successfully>

In [4]:
#pip install torchvision --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [5]:
lossF = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)


model.eval().to(device)
total_loss = 0
acc = 0

labelS = []
preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        results = model(inputs)
        loss = lossF(results, labels)
        total_loss += loss.item()
        _, predicted = torch.max(results, 1)
        acc += (predicted == labels).sum().item()


        labelS += labels.cpu().numpy().tolist()
        preds += predicted.cpu().numpy().tolist()


total_loss /= len(test_loader.dataset)
accuracy = acc / len(test_loader.dataset) * 100.0

print(f'Test Loss: {total_loss}, Test Accuracy: {accuracy}%')

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np
#Test verisinin Accuracy,
#Precision, Recall ve F1 skoru, Karmaşıklık Matrisi, Classification Report
print("Accuracy score : {}".format(accuracy_score(labelS,preds)))
print("Precision score : {}".format(precision_score(labelS,preds,average="weighted")))
print("Recall score : {} ".format(recall_score(labelS, preds, average='weighted')))
print("\nConfusion matrix :\n {}".format(confusion_matrix(labelS,preds)))
print("\nClassification Report : \n{}".format(classification_report(labelS,preds,target_names=[str(i) for i in range(len(dataset.classes))])))